# Diodenkennlinie Messung - Automatisierte Charakterisierung von Halbleiterdioden

## Einführung

Dieses Jupyter Notebook dokumentiert die automatisierte Messung der Kennlinie einer Halbleiterdiode. Die Diodenkennlinie zeigt den charakteristischen exponentiellen Zusammenhang zwischen Spannung und Strom in Durchlassrichtung und ist fundamental für das Verständnis von Halbleiterbauelementen.

### Theoretischer Hintergrund
Das Ziel ist die Messung der Strom-Spannungs-Kennlinie einer Diode, um eine einfache U-I-Kennlinie (U-I-Diagramm) aufzunehmen und darzustellen. Die Messung erfolgt durch schrittweises Anlegen einer Spannung (via DAC) und Messen des Stroms (via MCC118 und Shunt-Widerstand).
### Messmethodik

Wir verwenden eine **indirekte Strommessung** über einen Serienwiderstand:
- Spannungsquelle (DAC) → Serienwiderstand → Diode → Masse
- Strom berechnet nach: **I = (U_gesamt - U_diode) / R_serie**

---

## 1. Import der benötigten Bibliotheken

Die folgenden Bibliotheken werden für verschiedene Aspekte der Messung benötigt:

In [ ]:
from __future__ import print_function
import spidev          #SPI-Kommunikation mit DAC
import time            # Verzögerungen für Stabilisierung
import lgpio           # GPIO-Steuerung (moderne Alternative zu RPi.GPIO)
import matplotlib.pyplot as plt  # Visualisierung der Messergebnisse
from daqhats import mcc118, HatIDs  # MCC118 Datenerfassungssystem
from daqhats_utils import select_hat_device  # HAT-Geräte-Auswahl

### Bibliotheken-Übersicht:
- **spidev**: Ermöglicht SPI-Kommunikation mit dem Digital-Analog-Converter (DAC)
- **lgpio**: Moderne GPIO-Bibliothek für Chip Select Signal
- **daqhats**: Interface zum MCC118 für präzise Spannungsmessungen
- **matplotlib**: Erstellt die grafische Darstellung der Kennlinie

---

## 2. Systemkonfiguration und Hardware-Initialisierung

### 2.1 Konstanten und Parameter definieren

In [ ]:
CS_PIN = 22              # Chip Select Pin für SPI-DAC
MAX_DAC_VALUE = 4095     # 12-Bit DAC: 2^12 - 1 = 4095

# Globale Variable für GPIO-Handle
gpio_handle = None

**Erklärung der Konstanten:**
- **CS_PIN (22)**: GPIO-Pin für das Chip Select Signal des DAC
- **MAX_DAC_VALUE (4095)**: Maximaler digitaler Wert für 12-Bit DAC (entspricht maximaler Ausgangsspannung)

### 2.2 SPI-Interface Konfiguration

In [ ]:
# SPI-Schnittstelle für DAC-Kommunikation einrichten
spi = spidev.SpiDev()
spi.open(0, 0)              # SPI Bus 0, Device 0
spi.max_speed_hz = 1000000  # 1 MHz Übertragungsgeschwindigkeit
spi.mode = 0b00             # SPI Mode 0: CPOL=0, CPHA=0

**SPI-Parameter Erklärung:**
- **Bus 0, Device 0**: Erste SPI-Schnittstelle des Raspberry Pi
- **1 MHz**: Ausreichend schnell für DAC, aber stabil
- **Mode 0**: Clock Polarität und Phase für DAC-Kompatibilität

### 2.3 GPIO-Initialisierung

In [ ]:
# GPIO Setup mit lgpio (moderne, robuste GPIO-Bibliothek)
gpio_handle = lgpio.gpiochip_open(0)
if gpio_handle < 0:
    raise Exception("Fehler beim Öffnen des GPIO Chips")

# CS Pin als Ausgang konfigurieren
ret = lgpio.gpio_claim_output(gpio_handle, CS_PIN, lgpio.SET_PULL_NONE)
if ret < 0:
    raise Exception(f"Fehler beim Konfigurieren von GPIO Pin {CS_PIN}")

# CS Pin auf HIGH setzen (DAC nicht ausgewählt)
lgpio.gpio_write(gpio_handle, CS_PIN, 1)

**Warum lgpio statt RPi.GPIO?**
- Moderne, aktiv weiterentwickelte Bibliothek
- Bessere Fehlerbehandlung und Ressourcenverwaltung
- Kompatibel mit aktuellen Raspberry Pi OS Versionen

---

## 3. Funktionen für DAC-Steuerung und System-Management

### 3.1 DAC-Wert schreiben (write_dac)

Diese Funktion konvertiert einen digitalen Wert in eine entsprechende Ausgangsspannung:

In [ ]:
def write_dac(value):
    """DAC-Wert schreiben"""
    global gpio_handle
    assert 0 <= value <= MAX_DAC_VALUE
    control = 0
    control |= 0 << 15
    control |= 1 << 14
    control |= 0 << 13
    control |= 1 << 12
    data = control | (value & 0xFFF)
    high_byte = (data >> 8) & 0xFF
    low_byte = data & 0xFF
    lgpio.gpio_write(gpio_handle, CS_PIN, 0)
    spi.xfer2([high_byte, low_byte])
    lgpio.gpio_write(gpio_handle, CS_PIN, 1)

**Funktionsweise der DAC-Steuerung:**
- **Wertebereich prüfen**: Sicherstellen dass der Wert zwischen 0 und 4095 liegt
- **Kontrollbits setzen**: Die Bits 15-12 konfigurieren den DAC-Betrieb (Buffer, Gain, Shutdown, Channel)
- **Datenwort erstellen**: Kombination aus Kontrollbits und 12-Bit Wert
- **SPI-Übertragung**: Chip Select auf LOW → Daten senden → Chip Select auf HIGH

### 3.2 System-Cleanup Funktion

Sicheres Herunterfahren aller Hardware-Ressourcen:

In [ ]:
def cleanup_gpio():
    """GPIO-Cleanup"""
    global gpio_handle
    try:
        write_dac(0)
    except:
        pass
    try:
        if gpio_handle is not None:
            lgpio.gpio_free(gpio_handle, CS_PIN)
            lgpio.gpiochip_close(gpio_handle)
            gpio_handle = None
        spi.close()
        print("GPIO-Cleanup abgeschlossen")
    except Exception as e:
        print(f"Cleanup-Fehler: {e}")

**Warum ist Cleanup wichtig?**
- **DAC auf 0V setzen**: Sicherheit für nachfolgende Messungen
- **GPIO freigeben**: Verhindert Hardware-Locks beim nächsten Start
- **SPI schließen**: Gibt Systemressourcen frei

---

## 4. Hauptmessprogramm - Automatisierte Kennlinienaufnahme

### 4.1 Messsystem-Initialisierung und Kalibrierung

In [ ]:
def main():
    print("### Diodenkennlinie Messung ###\n")
    
    try:
        # MCC118 Verbindung
        address = select_hat_device(HatIDs.MCC_118)
        hat = mcc118(address)
        print(f"MCC 118 Gerät an Adresse {address} verbunden.")

**MCC118 Initialisierung:**
- **select_hat_device**: Automatische Erkennung des MCC118 HAT
- **mcc118(address)**: Erstellt Verbindung zum Messgerät
- Das MCC118 bietet 8 Analogeingänge mit 16-Bit Auflösung
### 4.2 System-Kalibrierung

In [ ]:
python
        # Kalibrierung maximale DAC-Spannung
        print("Kalibriere maximale DAC-Spannung an Kanal 7...")
        write_dac(MAX_DAC_VALUE)
        time.sleep(0.5)
        gemessene_max_spannung = hat.a_in_read(7)
        write_dac(0)
        print(f"Gemessene maximale Spannung (Referenz): {gemessene_max_spannung:.4f} V")

**Warum Kalibrierung notwendig?**
- **Hardware-Toleranzen**: DAC-Referenzspannung kann von nominellen 5V abweichen
- **Automatische Anpassung**: Alle weiteren Berechnungen basieren auf der tatsächlich gemessenen Maximalspannung
- **Messgenauigkeit**: Eliminiert systematische Fehler

### 4.3 Benutzer-Parameter erfassen

In [ ]:
 r_serie = float(input("Wert des Serienwiderstands in Ohm: "))
        anzahl_punkte = int(input("Anzahl der Spannungspunkte: "))

        spannung_max = float(input(f"Maximale Spannung in V (max {gemessene_max_spannung:.2f} V): "))
        if spannung_max > gemessene_max_spannung:
            print(f"Begrenze auf {gemessene_max_spannung:.2f} V.")
            spannung_max = gemessene_max_spannung

**Parameter-Bedeutung:**
- **r_serie**: Serienwiderstand zur Strommessung (typisch 100Ω - 10kΩ)
- **anzahl_punkte**: Mehr Punkte = höhere Auflösung, aber längere Messzeit
- **spannung_max**: Maximaler Spannungsbereich, begrenzt durch Hardware

### 4.4 DAC-Berechnung und Datenstrukturen

In [ ]:
# DAC-Wert, der genau spannung_max liefert
        dac_max_wert = int(MAX_DAC_VALUE * spannung_max / gemessene_max_spannung)

        # Listen für Messwerte
        eingestellte_spannungen = []
        diodenspannungen = []
        stroeme = []

**Berechnungslogik:**
- **dac_max_wert**: Umrechnung der gewünschten Maximalspannung in DAC-Stufen
- **Listen**: Separate Arrays für Sollwerte, Messwerte und berechnete Ströme

### 4.5 Automatisierte Messschleife

In [ ]:
print("\nMessung läuft...\n")

        for i in range(anzahl_punkte):
            # DAC linear von 0 bis dac_max_wert verteilen
            dac_value = int(i * dac_max_wert / (anzahl_punkte - 1))
            #print(dac_value)
            write_dac(dac_value)
            #print(dac_value)
            time.sleep(2.0)

            spannung_diode = hat.a_in_read(6)
            spannung_gesamt = hat.a_in_read(7)
            strom = ((spannung_gesamt - spannung_diode) / r_serie) * 1000  # mA

            # Geplante Spannung für Plot (0 bis spannung_max)
            spannung_dac = spannung_max * i / (anzahl_punkte - 1)
            print(spannung_dac)

            eingestellte_spannungen.append(spannung_dac)
            diodenspannungen.append(spannung_diode)
            stroeme.append(strom)

            print(f"Eingestellte Spannung: {spannung_dac:.3f} V | "
                  f"Gemessene Spannung: {spannung_gesamt:.5f} V | "
                  f"Diode: {spannung_diode:.5f} V | Strom: {strom:.3f} mA")

        write_dac(0)

**Messschleife-Erklärung:**
- **Lineare Verteilung**: DAC-Werte werden gleichmäßig von 0 bis Maximum verteilt
- **Stabilisierungszeit**: 2 Sekunden warten für elektrisches Einschwingen
- **Zwei Messungen**: Kanal 6 (Diode) und Kanal 7 (Gesamt) für Stromberechnung
- **Stromberechnung**: I = (U_gesamt - U_diode) / R_serie, Umrechnung in mA
- **Live-Ausgabe**: Sofortige Anzeige der Messwerte zur Kontrolle

### 4.6 Datenvisualisierung

In [ ]:
# Diagramm erstellen
        plt.figure(figsize=(12,5))

        plt.subplot(1,2,1)
        plt.plot(diodenspannungen, stroeme, marker='.')
        plt.xlabel("Spannung über Diode (V)")
        plt.ylabel("Strom durch Diode (mA)")
        plt.title("Diodenkennlinie")
        plt.grid(True)

        plt.subplot(1,2,2)
        plt.plot(eingestellte_spannungen, stroeme, marker='.', color='orange')
        plt.xlabel("Eingestellte Spannung (V)")
        plt.ylabel("Strom durch Diode (mA)")
        plt.title("Eingestellte Spannung vs. Strom")
        plt.xlim(0, spannung_max)
        plt.grid(True)

        plt.tight_layout()
        plt.show()

**Zwei-Diagramm-Ansatz:**
- **Linkes Diagramm**: Klassische I-U Diodenkennlinie (exponentieller Verlauf erwartet)
- **Rechtes Diagramm**: Systemverhalten (DAC-Eingangsspannung vs. resultierender Strom)
- **Unterschiedliche Perspektiven**: Hilft bei der Analyse von Messfehlern und Systemverhalten

### 4.7 Fehlerbehandlung

In [ ]:
except KeyboardInterrupt:
        print("Messung abgebrochen.")
    except Exception as e:
        print("Fehler:", e)
    finally:
        cleanup_gpio()

**Robuste Programmbeendigung:**
- **KeyboardInterrupt**: Sauberes Beenden bei Ctrl+C
- **Exception**: Allgemeine Fehlerbehandlung
- **finally**: Cleanup wird IMMER ausgeführt, auch bei Fehlern

---

## 5. Programmausführung

### 5.1 Hauptprogramm starten

In [ ]:
if __name__ == "__main__":
    main()

**Standard Python-Idiom:**
- Stellt sicher, dass main() nur beim direkten Ausführen des Scripts aufgerufen wird
- Nicht beim Import als Modul

---

## 6. Zusammenfassung

Dieses Programm realisiert eine vollautomatische Diodenkennlinie-Messstation mit:

**Technische Eigenschaften:**
- **12-Bit DAC-Auflösung**: 4096 diskrete Spannungswerte
- **16-Bit ADC-Genauigkeit**: Hochpräzise Spannungsmessungen  
- **Automatische Kalibrierung**: Kompensiert Hardware-Toleranzen
- **Robuste Fehlerbehandlung**: Sichere Hardware-Verwaltung

**Messtechnische Vorteile:**
- **Reproduzierbare Messungen**: Eliminiert menschliche Messfehler
- **Hohe Punktdichte**: Bis zu 1000+ Messpunkte möglich
- **Echzeit-Feedback**: Sofortige Kontrolle der Messwerte
- **Duale Visualisierung**: Verschiedene Analyseperspektiven

